<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
import spacy
nlp = spacy.load("en_core_web_lg")

In [2]:
%pwd

'/Users/andrewlowe/Lambda_Repositories/unit4/DS-Unit-4-Sprint-1-NLP/module4-topic-modeling'

In [3]:
path = '../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'

df = pd.read_csv(path)

In [8]:
df['reviews.text'].head()

0    I order 3 of them and one of the item is bad q...
1    Bulk is always the less expensive way to go fo...
2    Well they are not Duracell but for the price i...
3    Seem to work as well as name brand batteries a...
4    These batteries are very long lasting the pric...
Name: reviews.text, dtype: object

In [12]:
def tokenize(text):
    "Complete this function"
    
    return [token for token in simple_preprocess(text) if token in STOPWORDS]

In [30]:
df['tokens'] = df['reviews.text'].apply(tokenize)
df.tokens.shape

(28332,)

In [27]:
id2word = corpora.Dictionary(list(df['tokens']))

In [33]:
import sys
sys.getsizeof(id2word)

56

In [34]:
id2word.filter_extremes(no_below=5, no_above=0.95)


In [35]:
len(id2word.keys())

279

In [37]:
corpus = [id2word.doc2bow(text) for text in list(df['tokens'])]

In [39]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 1),
 (4, 3),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 2),
 (9, 1),
 (10, 2)]

In [40]:
lda = LdaMulticore(corpus=corpus,      # LDA multicore is the multiple distributed version of LDA. (like n_jobs on sklearn.)
                   id2word=id2word,     
                   random_state=723812,   # Random state not supre reliable in distributed model
                   num_topics = 15,     
                   passes=10,           # Num of times it tries to fit the distributions. (This has strongest effect of runtime). Maybe try 100 or 200?
                   workers=8     # Also number of distributed cores to multicore.
                  )

In [41]:
lda.print_topics()

[(0,
  '0.083*"the" + 0.081*"one" + 0.052*"my" + 0.050*"for" + 0.050*"so" + 0.049*"and" + 0.045*"this" + 0.040*"had" + 0.038*"fire" + 0.035*"no"'),
 (1,
  '0.259*"as" + 0.061*"the" + 0.058*"last" + 0.049*"just" + 0.044*"well" + 0.040*"name" + 0.038*"and" + 0.033*"these" + 0.028*"they" + 0.028*"other"'),
 (2,
  '0.123*"it" + 0.077*"for" + 0.074*"he" + 0.072*"and" + 0.065*"my" + 0.053*"was" + 0.049*"the" + 0.045*"this" + 0.043*"to" + 0.028*"his"'),
 (3,
  '0.094*"this" + 0.093*"the" + 0.093*"for" + 0.087*"is" + 0.086*"and" + 0.060*"my" + 0.059*"it" + 0.053*"has" + 0.048*"to" + 0.030*"that"'),
 (4,
  '0.176*"it" + 0.118*"for" + 0.089*"she" + 0.083*"my" + 0.059*"and" + 0.051*"this" + 0.042*"to" + 0.041*"her" + 0.025*"as" + 0.025*"so"'),
 (5,
  '0.114*"the" + 0.105*"very" + 0.081*"to" + 0.073*"of" + 0.060*"with" + 0.049*"and" + 0.044*"for" + 0.034*"they" + 0.027*"are" + 0.022*"in"'),
 (6,
  '0.142*"to" + 0.122*"it" + 0.087*"and" + 0.062*"my" + 0.042*"with" + 0.024*"the" + 0.024*"is" + 0.020

In [42]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]


In [43]:
topics = [' '.join(t[0:5]) for t in words]

In [44]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
the one my for so

------ Topic 1 ------
as the last just well

------ Topic 2 ------
it for he and my

------ Topic 3 ------
this the for is and

------ Topic 4 ------
it for she my and

------ Topic 5 ------
the very to of with

------ Topic 6 ------
to it and my with

------ Topic 7 ------
the to you it for

------ Topic 8 ------
for to it can and

------ Topic 9 ------
the is and it to

------ Topic 10 ------
the of in them and

------ Topic 11 ------
for we our and the

------ Topic 12 ------
the is can it you

------ Topic 13 ------
and to have it this

------ Topic 14 ------
they these and to are



In [45]:
import pyLDAvis.gensim    # Helps us visulize the topic models.

pyLDAvis.enable_notebook()

In [46]:
# Visualization of distance (similarity) of all these topics. size = size of topics in corpus. 

pyLDAvis.gensim.prepare(lda, corpus, id2word)

/Applications/anaconda3/envs/U4-S1/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.003569  0.023344       1        1  10.649240
10     0.073350  0.076435       2        1   9.639047
14     0.158536  0.016483       3        1   9.024669
9     -0.038291  0.059473       4        1   8.280502
12    -0.007271  0.041481       5        1   7.844942
3     -0.134776  0.011848       6        1   7.186439
4     -0.123294 -0.128715       7        1   6.106762
6     -0.104251  0.008866       8        1   5.929360
13    -0.028460  0.007867       9        1   5.903097
2     -0.118817 -0.026701      10        1   5.751521
1      0.260327 -0.061188      11        1   5.729122
0     -0.045002  0.063264      12        1   5.704442
8      0.010229 -0.019479      13        1   4.922853
5      0.049193  0.055476      14        1   4.035295
11     0.044959 -0.128455      15        1   3.292713, topic_info=    Category          Freq    Term         Total  loglift  logprob
22   Default   7731.000000      as   7731.000000  30.0000  30.0000
69   Default  19305.000000      it  19305.000000  29.0000  29.0000
12   Default  17974.000000     for  17974.000000  28.0000  28.0000
81   Default   2365.000000      we   2365.000000  27.0000  27.0000
173  Default   2638.000000     she   2638.000000  26.0000  26.0000
15   Default   4436.000000   these   4436.000000  25.0000  25.0000
2    Default  11605.000000      is  11605.000000  24.0000  24.0000
20   Default   5948.000000    they   5948.000000  23.0000  23.0000
27   Default   3873.000000    very   3873.000000  22.0000  22.0000
47   Default   4544.000000     you   4544.000000  21.0000  21.0000
8    Default  29198.000000     the  29198.000000  20.0000  20.0000
141  Default   1592.000000      he   1592.000000  19.0000  19.0000
70   Default  11260.000000      my  11260.000000  18.0000  18.0000
10   Default  19689.000000      to  19689.000000  17.0000  17.0000
5    Default   2992.000000     one   2992.000000  16.0000  16.0000
98   Default  10541.000000    this  10541.000000  15.0000  15.0000
9    Default   3213.000000    them   3213.000000  14.0000  14.0000
39   Default   2203.000000    last   2203.000000  13.0000  13.0000
55   Default   3909.000000     can   3909.000000  12.0000  12.0000
37   Default   6060.000000    with   6060.000000  11.0000  11.0000
110  Default    940.000000     our    940.000000  10.0000  10.0000
4    Default   8400.000000      of   8400.000000   9.0000   9.0000
160  Default   1531.000000     her   1531.000000   8.0000   8.0000
185  Default   2347.000000     has   2347.000000   7.0000   7.0000
0    Default  22552.000000     and  22552.000000   6.0000   6.0000
17   Default   4643.000000     are   4643.000000   5.0000   5.0000
25   Default   1101.000000    name   1101.000000   4.0000   4.0000
1    Default   5733.000000    have   5733.000000   3.0000   3.0000
206  Default   2425.000000    fire   2425.000000   2.0000   2.0000
84   Default   4125.000000     was   4125.000000   1.0000   1.0000
..       ...           ...     ...           ...      ...      ...
156  Topic15     31.730896  though    315.518188   1.1165  -5.9561
148  Topic15     22.197023   those    240.409653   1.0311  -6.3134
9    Topic15    289.116516    them   3213.224121   1.0053  -3.7466
138  Topic15     26.631338    both    364.844086   0.7961  -6.1313
80   Topic15     53.353298    many    788.916870   0.7197  -5.4364
124  Topic15     16.917284  almost    250.426071   0.7186  -6.5850
164  Topic15     14.534419    once    221.313171   0.6904  -6.7369
11   Topic15     33.510117  always    544.355530   0.6257  -5.9015
91   Topic15     47.633423    were    863.059631   0.5165  -5.5498
49   Topic15     87.491226      do   1589.316650   0.5139  -4.9418
37   Topic15    297.936768    with   6060.464844   0.4008  -3.7165
0    Topic15    841.065918     and  22552.128906   0.1245  -2.6787
17   Topic15    205.001129     are   4643.108398   0.2933  -4.0904
8    Topic15    611.3079

In [47]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=6,
                                                        passes=40)

In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.head()

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling